# Car Price prediction

<img src="https://whatcar.vn/media/2018/09/car-lot-940x470.jpg"/>

## Прогнозирование стоимости автомобиля по характеристикам
*Этот ноутбук является шаблоном (Baseline) к текущему соревнованию и не служит готовым решением!*   
Вы можете использовать его как основу для построения своего решения.


> **Baseline** создается больше как шаблон, где можно посмотреть, как происходит обращение с входящими данными и что нужно получить на выходе. При этом ML начинка может быть достаточно простой. Это помогает быстрее приступить к самому ML, а не тратить ценное время на инженерные задачи. 
Также baseline является хорошей опорной точкой по метрике. Если наше решение хуже baseline -  мы явно делаем что-то не так и стоит попробовать другой путь) 

## В baseline мы сделаем следующее:
* Построим "наивную"/baseline модель, предсказывающую цену по модели и году выпуска (с ней будем сравнивать другие модели)
* Обработаем и отнормируем признаки
* Сделаем первую модель на основе градиентного бустинга с помощью CatBoost
* Сделаем вторую модель на основе нейронных сетей и сравним результаты
* Сделаем multi-input нейронную сеть для анализа табличных данных и текста одновременно
* Добавим в multi-input сеть обработку изображений
* Осуществим ансамблирование градиентного бустинга и нейронной сети (усреднение их предсказаний)

In [1]:
!pip install -q tensorflow

In [ ]:
!pip install --upgrade pip

In [2]:
#аугментации изображений
!pip install albumentations -q

In [3]:
!pip install pymorphy2
!pip install pymorphy2-dicts
!pip install DAWG-Python

In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import random
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import sys
import PIL
import cv2
import re
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

#NLP
import nltk
import pymorphy2
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
nltk.download('stopwords')

# # keras
import tensorflow as tf
import tensorflow.keras.layers as L
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras import regularizers
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import albumentations

# plt
import matplotlib.pyplot as plt
#увеличим дефолтный размер графиков
from pylab import rcParams
rcParams['figure.figsize'] = 10, 5
#графики в svg выглядят более четкими
%config InlineBackend.figure_format = 'svg' 
%matplotlib inline

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
print('Python       :', sys.version.split('\n')[0])
print('Numpy        :', np.__version__)
print('Tensorflow   :', tf.__version__)

In [5]:
def mape(y_true, y_pred):
    return np.mean(np.abs((y_pred-y_true)/y_true))

In [6]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

In [7]:
!pip freeze > requirements.txt

# DATA

Посмотрим на типы признаков:

* bodyType - категориальный
* brand - категориальный
* color - категориальный
* description - текстовый
* engineDisplacement - числовой, представленный как текст
* enginePower - числовой, представленный как текст
* fuelType - категориальный
* mileage - числовой
* modelDate - числовой
* model_info - категориальный
* name - категориальный, желательно сократить размерность
* numberOfDoors - категориальный
* price - числовой, целевой
* productionDate - числовой
* sell_id - изображение (файл доступен по адресу, основанному на sell_id)
* vehicleConfiguration - не используется (комбинация других столбцов)
* vehicleTransmission - категориальный
* Владельцы - категориальный
* Владение - числовой, представленный как текст
* ПТС - категориальный
* Привод - категориальный
* Руль - категориальный

In [8]:
DATA_DIR = '../input/sf-dst-car-price-prediction-part2/'
train = pd.read_csv(DATA_DIR + 'train.csv')
test = pd.read_csv(DATA_DIR + 'test.csv')
sample_submission = pd.read_csv(DATA_DIR + 'sample_submission.csv')

In [9]:
train.info()

In [10]:
train.nunique()

# Model 1: Создадим "наивную" модель 
Эта модель будет предсказывать среднюю цену по модели и году выпуска. 
C ней будем сравнивать другие модели.



In [11]:
# split данных
data_train, data_test = train_test_split(train, test_size=0.15, shuffle=True, random_state=RANDOM_SEED)

In [12]:
# Наивная модель
predicts = []
for index, row in pd.DataFrame(data_test[['model_info', 'productionDate']]).iterrows():
    query = f"model_info == '{row[0]}' and productionDate == '{row[1]}'"
    predicts.append(data_train.query(query)['price'].median())

# заполним не найденные совпадения
predicts = pd.DataFrame(predicts)
predicts = predicts.fillna(predicts.median())

# округлим
predicts = (predicts // 1000) * 1000

#оцениваем точность
print(f"Точность наивной модели по метрике MAPE: {(mape(data_test['price'], predicts.values[:, 0]))*100:0.2f}%")

# EDA

Проведем быстрый анализ данных для того, чтобы понимать, сможет ли с этими данными работать наш алгоритм.

Посмотрим, как выглядят распределения числовых признаков:

In [13]:
#посмотрим, как выглядят распределения числовых признаков
def visualize_distributions(titles_values_dict):
  columns = min(3, len(titles_values_dict))
  rows = (len(titles_values_dict) - 1) // columns + 1
  fig = plt.figure(figsize = (columns * 6, rows * 4))
  for i, (title, values) in enumerate(titles_values_dict.items()):
    hist, bins = np.histogram(values, bins = 20)
    ax = fig.add_subplot(rows, columns, i + 1)
    ax.bar(bins[:-1], hist, width = (bins[1] - bins[0]) * 0.7)
    ax.set_title(title)
  plt.show()

visualize_distributions({
    'mileage': train['mileage'].dropna(),
    'modelDate': train['modelDate'].dropna(),
    'productionDate': train['productionDate'].dropna()
})

Итого:
* CatBoost сможет работать с признаками и в таком виде, но для нейросети нужны нормированные данные.

# PreProc Tabular Data

In [14]:
#используем все текстовые признаки как категориальные без предобработки
categorical_features = ['bodyType', 'brand', 'color', 'engineDisplacement', 'enginePower', 'fuelType', 'model_info', 'name',
  'numberOfDoors', 'vehicleTransmission', 'Владельцы', 'Владение', 'ПТС', 'Привод', 'Руль']

#используем все числовые признаки
numerical_features = ['mileage', 'modelDate', 'productionDate']

In [15]:
# ВАЖНО! дря корректной обработки признаков объединяем трейн и тест в один датасет
train['sample'] = 1 # помечаем где у нас трейн
test['sample'] = 0 # помечаем где у нас тест
test['price'] = 0 # в тесте у нас нет значения price, мы его должны предсказать, поэтому пока просто заполняем нулями

data = test.append(train, sort=False).reset_index(drop=True) # объединяем
print(train.shape, test.shape, data.shape)

In [16]:
data.head()

In [17]:
data.columns

In [18]:
# Пустые значения находятся столбце Владельцы и владение. Во владельцах заменим на медиану, а владении удалим, т.к. больше половины отсутствует
data.isna().sum()

In [19]:
# Переименуем названия столцов
data = data.rename(columns={'bodyType': 'body_Type', 'engineDisplacement': 'engine_Displacement', 'enginePower': 'engine_Power', 'fuelType': 'fuel_Type', 'mileage': 'mile_age', 'modelDate': 'model_Date', 'numberOfDoors': 'number_Of_Doors', 'productionDate': 'production_Date', 'vehicleConfiguration': 'vehicle_Configuration', 'vehicleTransmission': 'vehicle_Transmission', 'Владельцы':'owners', 'Владение':'possession', 'ПТС':'PTS', 'Привод': 'drive', 'Руль': 'drive_side'})

In [20]:
data.head()

## Предобработка табличных данных

In [21]:
# Найдем количество дубликатов, если такие имеются

data.groupby(['sell_id']).size().sort_values()

## Body_Type

In [24]:
data['body_Type'].unique()

In [25]:
data['body_Type'].hist(xrot=90)

In [26]:
# Больше всего машин предствалено с типом кузова седан и внедорожник 5дв. в одном экземпляре присутствует седан 2дв.
data['body_Type'].value_counts()

In [27]:
# Визуализируем авто с 2 дверьми
image = PIL.Image.open('../input/sf-dst-car-price-prediction-part2/img/img/'+
                       str(data[data['body_Type']== 'седан 2 дв.'].sell_id.values[0])+
                       '.jpg')
imgplot = plt.imshow(image)
plt.show()
image.size

## Brand

In [28]:
# В данном датасете автомобили в 3 моделях.
data['brand'].hist()

In [29]:
data['brand'].value_counts()

## Color

In [30]:
data['color'].value_counts()

## engine_Displacement

In [31]:
# 
data['engine_Displacement'].value_counts()

In [61]:
# представим его в числовой, убрав LTR
data['engine_Displacement'] = data['engine_Displacement'].apply(lambda x: x[:-3])
data['engine_Displacement'] = data['engine_Displacement'].apply(pd.to_numeric, errors='coerce')
data['engine_Displacement'].unique()

In [32]:
data['engine_Displacement'].unique()

## engine_Power

In [33]:
data['engine_Power'].unique()

In [ ]:
#data['engine_Power'] = data['engine_Power'].apply(lambda x: x[:-3])
#data['engine_Power'] = data['engine_Power'].apply(pd.to_numeric, errors='coerce')

In [34]:
data['engine_Power'].hist()
data['engine_Power'].value_counts()

In [35]:
# По типу топлива автомобили в основном на бензине. Есть несколько авто с типом топлива: гибрид и электро 
data['fuel_Type'].hist()
data['fuel_Type'].value_counts()

## Mille_age

In [36]:
data['mile_age'].hist()
data['mile_age'].value_counts()[:20]

In [37]:
data[data['mile_age'] > 200000]

## Model_Date

In [38]:
# Большая часть автомобилей с датой модели между 2005 и 2018годами
#data['model_Date'].hist()
data['model_Date'].value_counts().plot.barh()

In [39]:
data['model_Date'].value_counts()[:15]

## Model_info

In [41]:
# Данный столбец никаой информации нам не несет. Думаю можно удалить его
data['model_info'].unique()

## Name

In [42]:
# Данный столбец собержит значения которые имеются в других столбцам. Можно попробовать вытянуть какие то отдельные слова. В дальнейшем его удалить
data['name']

## number_Of_Doors

In [43]:
# по количеству дверей автомобили с 5, 4, 3, и 2 дверьми. Больше всего с 5 и 4 дверьми.
data['number_Of_Doors'].value_counts().plot.bar()
data['number_Of_Doors'].value_counts()

## Production_date

In [44]:
# Основная масса автомобилей с 2008 по 2018
data['production_Date'].value_counts()[:15]

## Sell_id

In [45]:
# данный столбец не несет никакой информации, его можно удалить
data['sell_id']

## vehicle_Configuration

In [46]:
# Значения данного столбца есть в столбцам. Нового он ничего нам не дает, его можно удалить
data['vehicle_Configuration']

## owners

In [47]:
# Столбец содержит информацию о количестве владельцев. Автомобили имеют 1, 2, 3 или более владельцев, также имеются значения NaN 
data['owners'].unique()

In [48]:
data['owners'].hist()
data['owners'].value_counts()

## possession

In [49]:
# Данный столбец имеет больше половины значений Nan, удалим его 
data['possession'].isna().sum()

## PTS

In [50]:
#Столбец с ПТС заполнен в основном оригиналами, пустых значений нет
data['PTS'].unique()
data['PTS'].value_counts()
data['PTS'].hist()

In [51]:
# Больше половины машин с полным приводом, с передним меньше всего
data['drive'].unique()
data['drive'].hist()
data['drive'].value_counts()

## drive_side

In [52]:
# С правым расположениме руля всего две машины
data['drive_side'].unique()
data['drive_side'].hist()
data['drive_side'].value_counts()

In [53]:
# Смотрим, что на картинку предполагаемого выброса в train
image = PIL.Image.open('../input/sf-dst-car-price-prediction-part2/img/img/'+
                       str(data[data['drive_side']=='Правый'].sell_id.values[0])+
                       '.jpg')
imgplot = plt.imshow(image)
plt.show()
image.size

In [54]:
#используем все текстовые признаки как категориальные без предобработки
categorical_features = ['body_Type', 'brand', 'color', 'fuel_Type', 'model_info', 'name', 'vehicle_Transmission', 'PTS', 'drive', 'drive_side', 'number_Of_Doors', 'owners']

#используем все числовые признаки
numerical_features = ['mile_age', 'model_Date', 'production_Date', 'engine_Displacement']

In [55]:
# Корреляция заметна между годом модели и годом выпуска авто, удалим столбец год модели. Также наблюдается коррелация между годом выпуска авто и пробегом, что вполне логично.
#plt.figure(figsize=(10,5))
sns.heatmap(data[numerical_features + categorical_features].corr(), annot=True)

## Проверим на выбросы

In [59]:
numerical_features

In [62]:
for col in numerical_features:
    IQR = data[col].quantile(0.75) - data[col].quantile(0.25)
    perc25 = data[col].quantile(0.25)  # 25-й перцентиль
    perc75 = data[col].quantile(0.75)  # 75-й перцентиль

    print(
        f'Название столбца:{col} '
        '25-й перцентиль: {},'.format(perc25),
        '75-й перцентиль: {},'.format(perc75),
        "IQR: {}, ".format(IQR),
        "Границы выбросов: [{f}, {l}].".format(f=perc25 - 1.5*IQR,
                                               l=perc75 + 1.5*IQR))

In [ ]:
def preproc_data_nn(df_input):
    '''includes several functions to pre-process the predictor data.'''
    
    df_output = df_input.copy()
    
    # ################### 1. Предобработка ############################################################## 
    # убираем не нужные для модели признаки
    df_output.drop(['description','sell_id'], axis = 1, inplace=True)
    
    
    # ################### Numerical Features ############################################################## 
    # Далее заполняем пропуски
    #for column in numerical_features:
     #   df_output[column].fillna(df_output[column].median(), inplace=True)
      #  df_output['owners'].fillna(df_output['owners'].median, inplace=True)
    # тут ваш код по обработке NAN
    # ....
    
    df_output['owners'] = df_output['owners'].apply(str)
    df_output['owners'] = df_output['owners'].apply(lambda x: x[:1])
    df_output['owners'] = df_output['owners'].apply(pd.to_numeric, errors='coerce')
    df_output['owners'].fillna(df_output['owners'].median(), inplace=True)
    
    #df_output['engine_Displacement'] = df_output['engine_Displacement'].apply(lambda x: x[:-3])
    #df_output['engine_Displacement'] = df_output['engine_Displacement'].apply(pd.to_numeric, errors='coerce')
    
    df_output['engine_Power'] = df_output['engine_Power'].apply(lambda x: x[:-3])
    df_output['engine_Power'] = np.log(df_output['engine_Power'].apply(pd.to_numeric, errors='coerce'))
    
       
    # Далее заполняем пропуски
    for column in numerical_features:
        df_output[column].fillna(df_output[column].mean(), inplace=True)
        
    # Логарифмируем числовые значения
    #for column in numerical_features:
    #    df_output[column] = np.log(df_output[column])
    
    
    # Нормализация данных
    scaler = MinMaxScaler()
    for column in numerical_features:
        df_output[column] = scaler.fit_transform(df_output[[column]])[:,0]
    
    
    
    # ################### Categorical Features ############################################################## 
    # Label Encoding
    for column in categorical_features:
        df_output[column] = df_output[column].astype('category').cat.codes
        
    # One-Hot Encoding: в pandas есть готовая функция - get_dummies.
    df_output = pd.get_dummies(df_output, columns=categorical_features, dummy_na=False)
    # тут ваш код не Encoding фитчей
        
    # ################### Feature Engineering ####################################################
    # создадим новый признак с автомобилями старше 1990 года
    #df_output['retro'] = df_output['model_Date'].apply(lambda x:1990 - x)
    #df_output['retro'] = df_output['retro'].apply(lambda x: 1 if x > 0 else 0)
    
    #df_output['owners_new'] = df_output['owners'].apply(lambda x: 1 if x == 3 else 0)
    
    # Признак возраста автомобиля (найдем разницу между датой модели и датой выпуска автомобиля)
    #df_output['age_auto'] = df_output['model_Date'] - df_output['production_Date']
    
    
    #df_output['age'] = np.log(2020 - df_output['production_Date'])
    df_output['xDrive'] = df_output['name'].apply(lambda x: 1 if 'xDrive' in x else 0)
    df_output['4WD'] = df_output['name'].apply(lambda x: 1 if '4WD' in x else 0)
    #df_output['model_prod'] = df_output['production_Date'] - df_output['model_Date']
    
    # ################### Clean #################################################### 
    # убираем признаки которые еще не успели обработать, 
    #df_output.drop(['vehicle_Configuration', 'possession', 'owners', 'mile_age'], axis = 1, inplace=True)
    df_output.drop(['vehicle_Configuration', 'possession', 'name', 'PTS', 'drive_side', 'color'], axis = 1, inplace=True)
    
    return df_output

In [ ]:
# Запускаем и проверяем, что получилось
df_preproc2 = preproc_data_nn(data)
df_preproc2.sample(3)

## Split data

In [65]:
# Теперь выделим тестовую часть
train_data = df_preproc2.query('sample == 1').drop(['sample'], axis=1)
test_data = df_preproc2.query('sample == 0').drop(['sample'], axis=1)

y = train_data.price.values     # наш таргет
X = train_data.drop(['price'], axis=1)
X_sub = test_data.drop(['price'], axis=1)

In [66]:
test_data.info()

# Model 2: CatBoostRegressor

In [67]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.14, shuffle=True, random_state=RANDOM_SEED)

In [76]:
model = CatBoostRegressor(iterations = 20000,
                          depth=10,
                          learning_rate = 0.002,
                          random_seed = RANDOM_SEED,
                          eval_metric='MAPE',
                          custom_metric=['RMSE', 'MAE'],
                          od_wait=500,
                          #task_type='GPU',
                         )
model.fit(X_train, np.log(y_train),
         eval_set=(X_test, np.log(y_test)),
         verbose_eval=300,
         use_best_model=True,
         plot=True
         )

In [77]:
test_predict_catboost = np.exp(model.predict(X_test))
print(f"TEST mape: {(mape(y_test, test_predict_catboost))*100:0.2f}%")

### Submission

In [78]:
sub_predict_catboost = model.predict(X_sub)
sample_submission['price'] = sub_predict_catboost
sample_submission.to_csv('catboost_submission.csv', index=False)

После обработки на Catboost получал результат в 11.09%.

# Model 3: Tabular NN

Построим обычную сеть:

In [69]:
#используем все текстовые признаки как категориальные без предобработки
categorical_features = ['body_Type', 'brand', 'fuel_Type', 'model_info', 'vehicle_Transmission', 'drive']
#categorical_features = ['brand', 'vehicle_Transmission', 'color', 'PTS', 'drive', 'model_info', 'body_Type', 'fuel_Type', 'drive_side', 'owners_new', 'mill_auto', 'number_Of_Doors']
#используем все числовые признаки
numerical_features = ['mile_age', 'model_Date', 'production_Date', 'engine_Displacement']

In [73]:
# Запускаем и проверяем, что получилось
df_preproc2 = preproc_data_nn(data)
df_preproc2.sample(3)

In [79]:
# Выделим тестовую часть
train_data_3 = df_preproc2.query('sample == 1').drop(['sample'], axis=1)
test_data_3 = df_preproc2.query('sample == 0').drop(['sample'], axis=1)

y = train_data_3.price.values     # наш таргет
X = train_data_3.drop(['price'], axis=1)
X_sub = test_data_3.drop(['price'], axis=1)

In [80]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.14, shuffle=True, random_state=RANDOM_SEED)

## Simple Dense NN

In [ ]:
#from tensorflow import keras 
#from keras.layers.advanced_activations import PReLU
#from tensorflow.keras.activations import elu
#para_relu = PReLU()
#model_1 = Sequential()
#model_1.add(L.Dense(512, input_dim=X_train.shape[1], activation='relu'))
#model_1.add(L.Dropout(0.5))
#model_1.add(L.Dense(256, activation="relu"))
#model_1.add(L.Dropout(0.5))
#model_1.add(L.Dense(1, activation="linear"))

In [ ]:
!pip install keras

In [ ]:
# для GPU
!pip install --upgrade tensorflow
!pip install --upgrade tensorflow-gpu

In [81]:
from tensorflow import keras 
from keras.layers.advanced_activations import PReLU
from tensorflow.keras.activations import elu
para_relu = PReLU()
model_1 = Sequential()
model_1.add(L.Dense(512, input_dim=X_train.shape[1], activation=para_relu))
model_1.add(L.Dropout(0.5))
model_1.add(L.Dense(256, activation="relu"))
model_1.add(L.Dropout(0.5))
model_1.add(L.Dense(1, activation="linear"))

In [ ]:
"""model = Sequential()
model.add(L.Dense(512, input_dim=X_train.shape[1], 
                  activation="sigmoid",
#                   activation='relu',
                  kernel_regularizer=regularizers.l2(l2=1e-6),
                  bias_regularizer=regularizers.l2(l2=1e-6),
                 ))
model.add(L.Dropout(0.5))
model.add(L.Dense(256, kernel_regularizer=regularizers.l2(l2=1e-5),
                  bias_regularizer=regularizers.l2(l2=1e-5),
                  activation="relu"))
model.add(L.Dropout(0.5))
model.add(L.Dense(256, kernel_regularizer=regularizers.l2(l2=0.00001), 
                  bias_regularizer=regularizers.l2(l2=0.00001),
                  activation="relu"))
model.add(L.Dropout(0.5))
model.add(L.Dense(32, kernel_regularizer=regularizers.l2(l2=0.001),
                  bias_regularizer=regularizers.l2(l2=0.001),
                  activation="relu"))
# model.add(L.Dropout(0.25))
model.add(L.Dense(1, activation="linear"))

In [ ]:
model_1.summary()

In [83]:
# Compile model
optimizer = tf.keras.optimizers.Adam(0.05) # 0.01
model_1.compile(loss='MAPE',optimizer=optimizer, metrics=['MAPE'])

In [84]:
checkpoint = ModelCheckpoint('../working/best_model.hdf5' , monitor=['val_MAPE'], verbose=0  , mode='min')
earlystop = EarlyStopping(monitor='val_MAPE', patience=150, restore_best_weights=True,)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.6,
                              patience=40, min_lr=0.00001)
callbacks_list = [checkpoint, earlystop, reduce_lr]

### Fit

In [85]:
history = model_1.fit(X_train, y_train,
                    batch_size=512,
                    epochs=2500, # фактически мы обучаем пока EarlyStopping не остановит обучение
                    validation_data=(X_test, y_test),
                    callbacks=callbacks_list,
                    verbose=2,
                   )

In [86]:
plt.title('Loss')
plt.plot(history.history['MAPE'], label='train')
plt.plot(history.history['val_MAPE'], label='test')
plt.show();

In [87]:
model_1.load_weights('../working/best_model.hdf5')
model_1.save('../working/nn_1.hdf5')

In [88]:
test_predict_nn1 = model_1.predict(X_test)
print(f"TEST mape: {(mape(y_test, test_predict_nn1[:,0]))*100:0.2f}%")

In [90]:
sub_predict_nn1 = model_1.predict(X_sub)
sample_submission['price'] = sub_predict_nn1[:,0]
sample_submission.to_csv('nn1_submission.csv', index=False)

Пробовал логарифмировать признаки, но в итоге с МинМакс дало больший результат. Уменьшал скорость, пробовал увеличивать количество эпох, размер батча. Не много дало прирост при смене активатора слоя модели. Уменьшил немного размерность категориальных признаков, тоже дало результат. Результат получился не много чухе чем с CatBoost

# Model 4: NLP + Multiple Inputs

In [91]:
data.description

In [92]:
# TOKENIZER
# The maximum number of words to be used. (most frequent)
MAX_WORDS = 100000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 256

In [ ]:
#tokenize.word_index

## Stopworld

In [93]:
# Посмотрим на состав имеющихся стоп-слов
stopwords_ru = stopwords.words("russian")
print(stopwords.words("russian"))

In [94]:
# Теперь выведем все слова длиной менее 4-х символов и дополним список стоп-слов
stopworlds_new = set()
mas_stop = set()
for words in data.description:
    for i in words.split():
        if len(i) <= 4:
            mas_stop.add(i)
 
stopworlds_new = set(stopwords_ru).union(mas_stop)
print(stopworlds_new)
#print(mas_stop)

## Lemmatizer

In [95]:
# сделаем функцию, в котороой очистим текст от знаков пунктуации и др.символов, далее посплитим на слова, удалим пробелы до и после, приведем слова к первоначальной форме. Также в конце удалим 10 слов часто встречающиеся.    
stemmer = SnowballStemmer("russian") 
morph = pymorphy2.MorphAnalyzer()

patterns = "[A-Za-z0-9!#$%&'()*+,./:“″;”<=>?@[\]^_`{|}~—\"\-•–«»]+"
def lemmatize(doc):
    doc = doc.lower()
    tokens = []
    doc = re.sub(patterns, ' ', doc)
    for token in doc.split():
        if token and token not in stopworlds_new:
            token = token.strip()
            token = morph.normal_forms(token)[0]  # Лемматизация
            #token = stemmer.stem(token) # Стеммизация
            tokens.append(token)
 #   if len(tokens) >=0 :
     #   tokens = [word for word in tokens if not word in worlds_freq_2]
    return ' '.join(tokens) 
  #  return None

In [96]:
# Применим функцию к нашему столбцу с данными, при этом создав новый столбец
data['description_clean'] = data.description.apply(lambda x: lemmatize(x))

## Частотность слов

In [97]:
# Сначала выведим список слов которые часто встречаются
words =list( data['decription_clean'].values)

split_words=[]
for word in words:
    lo_w=[]
    list_of_words=str(word).split()
    for w in list_of_words:
        if w not in stopworlds_new:
            lo_w.append(w)
    split_words.append(lo_w)
allwords = []
for wordlist in split_words:
    allwords += wordlist

In [98]:
# Получим 20 часто встречающихся слов
from nltk import FreqDist
word_freq = FreqDist(allwords).most_common(20)
word_freq

In [100]:
# создадим список из 15 слов наиболее часто встречающихся
worlds_freq_2 = ['автомобиль', 'система', 'пробег', 'задний', 'салон', 'сидение', 'официальный', 'передний', 'машина', 'комплект', 'дилер', 'новый', 'официальный', 'кредит', 'гарантия']

In [101]:
# Добавим в уже существующуюся функцию, исключение частовстречающихся слов из списка

patterns = "[A-Za-z0-9!#$%&'()*+,./:“″;”<=>?@[\]^_`{|}~—\"\-•–«»]+"
def lemmatize(doc):
    doc = doc.lower()
    tokens = []
    doc = re.sub(patterns, ' ', doc)
    for token in doc.split():
        if token and token not in stopworlds_new:
            token = token.strip()
            token = morph.normal_forms(token)[0]  # Лемматизация
            #token = stemmer.stem(token) # Стеммизация
            tokens.append(token)
    if len(tokens) >=0 :
        tokens = [word for word in tokens if not word in worlds_freq_2]
    return ' '.join(tokens) 

In [106]:
# Создадим новый столбец, без слов из списка часто встречающихся
data['description_clean'] = data.description.apply(lambda x: lemmatize(x))

# BERT

In [ ]:
!pip install transformers
import transformers as ppb

In [ ]:
# For DistilBERT:
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

## Want BERT instead of distilBERT? Uncomment the following line:
#model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

In [ ]:
tokenized = data['description'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

In [ ]:
tokenized

In [ ]:
# Прокладка
# После токенизации tokenized представляет собой список предложений — каждое предложение представлено в виде списка токенов. 
# Мы хотим, чтобы BERT обрабатывал все наши примеры одновременно (как один пакет). Просто так быстрее. 
# По этой причине нам нужно дополнить все списки до одинакового размера, чтобы мы могли представить ввод как один двумерный массив, а не список списков (разной длины).
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

In [ ]:
# Наш набор данных теперь находится в дополненной переменной, мы можем просмотреть его размеры ниже:

np.array(padded).shape

In [ ]:
# Маскировка
# Если мы напрямую отправим padded в BERT, это немного запутает его. Нам нужно создать еще одну переменную, 
# чтобы указать ей игнорировать (маскировать) заполнение, которое мы добавили, когда она обрабатывает свой ввод. 
# Вот что такое attribute_mask:

attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

In [ ]:
# Запустим обучение модели
# Функция model() запускает наши предложения через BERT. Результаты обработки будут возвращены в last_hidden_states

import torch
input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [ ]:
# Давайте нарежем только ту часть вывода, которая нам нужна. Это вывод, соответствующий первому токену каждого предложения. 
# Способ, которым BERT выполняет классификацию предложений, заключается в том, что он добавляет токен с именем [CLS] (для классификации) в начале каждого предложения. 
# Вывод, соответствующий этому токену, можно рассматривать как вложение всего предложения.
# Мы сохраним их в переменной функций, поскольку они будут служить функциями нашей модели логической регрессии.

features = last_hidden_states[0][:,0,:].numpy()

In [ ]:
labels = batch_1[1]

In [109]:
# Посмотрим что было и что получилось
print(data['description_clean'][13])
print("_______________________________")
print(data['description'][13])

In [110]:
# split данных
text_train = data.description_clean.iloc[X_train.index]
text_test = data.description_clean.iloc[X_test.index]
text_sub = data.description_clean.iloc[X_sub.index]

In [111]:
%%time
tokenize = Tokenizer(num_words=MAX_WORDS)
tokenize.fit_on_texts(data.decription_clean)

In [112]:
%%time
text_train_sequences = sequence.pad_sequences(tokenize.texts_to_sequences(text_train), maxlen=MAX_SEQUENCE_LENGTH)
text_test_sequences = sequence.pad_sequences(tokenize.texts_to_sequences(text_test), maxlen=MAX_SEQUENCE_LENGTH)
text_sub_sequences = sequence.pad_sequences(tokenize.texts_to_sequences(text_sub), maxlen=MAX_SEQUENCE_LENGTH)

print(text_train_sequences.shape, text_test_sequences.shape, text_sub_sequences.shape, )

In [ ]:
# вот так теперь выглядит наш текст
print(text_train.iloc[16])
print(text_train_sequences[16])

### RNN NLP

In [113]:
model_nlp = Sequential()
model_nlp.add(L.Input(shape=MAX_SEQUENCE_LENGTH, name="seq_description"))
model_nlp.add(L.Embedding(len(tokenize.word_index)+1, MAX_SEQUENCE_LENGTH,))
model_nlp.add(L.LSTM(256, return_sequences=True))
model_nlp.add(L.Dropout(0.5))
model_nlp.add(L.LSTM(128,))
model_nlp.add(L.Dropout(0.25))
model_nlp.add(L.Dense(64, activation="relu"))
model_nlp.add(L.Dropout(0.25))

### MLP

In [114]:
# Модель из предыдущего блока

from keras.layers.advanced_activations import PReLU
from tensorflow.keras.activations import elu
para_relu = PReLU()
model_mlp = Sequential()
model_mlp.add(L.Dense(512, input_dim=X_train.shape[1], activation=para_relu))
model_mlp.add(L.Dropout(0.5))
model_mlp.add(L.Dense(256, activation="relu"))
model_mlp.add(L.Dropout(0.5))
model_mlp.add(L.Dense(1, activation="linear"))

### Multiple Inputs NN

In [115]:
combinedInput = L.concatenate([model_nlp.output, model_mlp.output])
# being our regression head
head = L.Dense(64, activation="relu")(combinedInput)
head = L.Dense(1, activation="linear")(head)

model_2 = Model(inputs=[model_nlp.input, model_mlp.input], outputs=head)

In [116]:
model_2.summary()

### Fit

In [117]:
optimizer = tf.keras.optimizers.Adam(0.05) # 0.05
model_2.compile(loss='MAPE',optimizer=optimizer, metrics=['MAPE'])

In [ ]:
checkpoint = ModelCheckpoint('../working/best_model_1.hdf5', monitor=['val_MAPE'], verbose=0, mode='min')
earlystop = EarlyStopping(monitor='val_MAPE', patience=200, restore_best_weights=True,)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5,
                              patience=100, min_lr=0.0001)
callbacks_list = [checkpoint, earlystop, reduce_lr]

In [ ]:
history = model_2.fit([text_train_sequences, X_train], y_train,
                    batch_size=512,
                    epochs=2000, # фактически мы обучаем пока EarlyStopping не остановит обучение
                    validation_data=([text_test_sequences, X_test], y_test),
                    callbacks=callbacks_list
                   )

In [ ]:
plt.title('Loss')
plt.plot(history.history['MAPE'], label='train')
plt.plot(history.history['val_MAPE'], label='test')
plt.show();

In [ ]:
model_2.load_weights('../working/best_model_1.hdf5')
model_2.save('../working/nn_2.hdf5')

In [ ]:
test_predict_nn2 = model_2.predict([text_test_sequences, X_test])
print(f"TEST mape: {(mape(y_test, test_predict_nn2[:,0]))*100:0.2f}%")

Пробовал, эксперименты сл скоростью, увеличивать уменьшать список частовстречающихся слов а также стоп-слов. Также попробовал прикрутить Bert, но пока нет понимания как она тут должна работать. Сколько примеров я посмотрел, Bert используют для классификации. Результат после очистки текста и удаления стоп 10.60%

In [ ]:
sub_predict_nn2 = model_2.predict([text_sub_sequences, X_sub])
sample_submission['price'] = sub_predict_nn2[:,0]
sample_submission.to_csv('nn2_submission1.csv', index=False)

# Model 5: Добавляем картинки

### Data

In [118]:
# убедимся, что цены и фото подгрузились верно
plt.figure(figsize = (12,8))

random_image = train.sample(n = 9)
random_image_paths = random_image['sell_id'].values
random_image_cat = random_image['price'].values

for index, path in enumerate(random_image_paths):
    im = PIL.Image.open(DATA_DIR+'img/img/' + str(path) + '.jpg')
    plt.subplot(3, 3, index + 1)
    plt.imshow(im)
    plt.title('price: ' + str(random_image_cat[index]))
    plt.axis('off')
plt.show()

In [119]:
#size = (320, 240)
size = (320, 280)

def get_image_array(index):
    images_train = []
    for index, sell_id in enumerate(data['sell_id'].iloc[index].values):
        image = cv2.imread(DATA_DIR + 'img/img/' + str(sell_id) + '.jpg')
        assert(image is not None)
        image = cv2.resize(image, size)
        images_train.append(image)
    images_train = np.array(images_train)
    print('images shape', images_train.shape, 'dtype', images_train.dtype)
    return(images_train)

images_train = get_image_array(X_train.index)
images_test = get_image_array(X_test.index)
images_sub = get_image_array(X_sub.index)

### albumentations

In [120]:
from albumentations import (
    HorizontalFlip, IAAPerspective, ShiftScaleRotate, CLAHE, RandomRotate90,
    Transpose, ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion, HueSaturationValue,
    IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, MedianBlur, IAAPiecewiseAffine,
    IAASharpen, IAAEmboss, RandomBrightnessContrast, Flip, OneOf, Compose
)


#пример взят из официальной документации: https://albumentations.readthedocs.io/en/latest/examples.html
augmentation_1 = Compose([
    HorizontalFlip(p=0.4),
    OneOf([
        IAAAdditiveGaussianNoise(p=0.4),
        GaussNoise(p=0.8),
    ], p=0.2),
    OneOf([
        MotionBlur(p=0.2),
        MedianBlur(blur_limit=3, p=0.2),
        Blur(blur_limit=3, p=0.3),
    ], p=0.2),
    ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.3, rotate_limit=10, p=1),
    OneOf([
        OpticalDistortion(p=0.1),
        GridDistortion(p=0.3),
        #PiecewiseAffine(p=0.1),
    ], p=0.2),
    OneOf([
        CLAHE(clip_limit=5), # Примените адаптивное выравнивание гистограммы с ограниченным контрастом к входному изображению.
        IAASharpen(p=0.3), # Увеличьте резкость входного изображения и наложите результат на исходное изображение.
        IAAEmboss(), # Тиснение входного изображения и наложение результата на исходное изображение.
        RandomBrightnessContrast(), # Произвольное изменение яркости и контрастности входного изображения.
    ], p=0.3),
    HueSaturationValue(p=0.2), # Произвольное изменение оттенка, насыщенности и значения входного изображения.
], p=1)

#пример
plt.figure(figsize = (12,8))
for i in range(9):
    img = augmentation_1(image = images_train[20])['image']
    plt.subplot(3, 3, i + 1)
    plt.imshow(img)
    plt.axis('off')
plt.show()

In [121]:
# аугментация_2

augmentation_2 = Compose([
    HorizontalFlip(p=0.1),
    OneOf([
        IAAAdditiveGaussianNoise(p=0.8),
        GaussNoise(p=0.8),
    ], p=0.2),
    
    ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.4, rotate_limit=5, p=1),
    OneOf([
        OpticalDistortion(p=0.2),
        GridDistortion(p=0.2),
        #PiecewiseAffine(p=0.1),
    ], p=0.2),
    OneOf([
        CLAHE(clip_limit=5), # Примените адаптивное выравнивание гистограммы с ограниченным контрастом к входному изображению.
        IAASharpen(p=0.8), # Увеличьте резкость входного изображения и наложите результат на исходное изображение.
        IAAEmboss(p=0.8), # Тиснение входного изображения и наложение результата на исходное изображение.
        RandomBrightnessContrast(), # Произвольное изменение яркости и контрастности входного изображения.
    ], p=0.3),
    HueSaturationValue(p=0.2), # Произвольное изменение оттенка, насыщенности и значения входного изображения.
], p=1)

#пример
plt.figure(figsize = (12,10))
for i in range(9):
    img = augmentation_2(image = images_train[20])['image']
    plt.subplot(3, 3, i + 1)
    plt.imshow(img)
    plt.axis('off')
plt.show()

In [122]:
# аугментация_3

augmentation_3 = Compose([
    HorizontalFlip(p=0.4),
    OneOf([
        IAAAdditiveGaussianNoise(p=0.1), # Добавьте гауссовский шум к входному изображению
        GaussNoise(p=0.2),
    ], p=0.2),
    OneOf([
        MotionBlur(p=0.2), # Примените размытие движения к входному изображению с помощью ядра случайного размера
        MedianBlur(blur_limit=3, p=0.1),
        Blur(blur_limit=3, p=0.3),
    ], p=0.2),
    ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.1, rotate_limit=5, p=1), # Произвольно применяйте аффинные преобразования: переводите, масштабируйте и поворачивайте ввод
    OneOf([
        OpticalDistortion(p=0.1),
        GridDistortion(p=0.2),
        #PiecewiseAffine(p=0.1),
    ], p=0.2),
    OneOf([
        CLAHE(clip_limit=5), # Примените адаптивное выравнивание гистограммы с ограниченным контрастом к входному изображению.
        IAASharpen(p=0.1), # Увеличьте резкость входного изображения и наложите результат на исходное изображение.
        IAAEmboss(), # Тиснение входного изображения и наложение результата на исходное изображение.
        RandomBrightnessContrast(), # Произвольное изменение яркости и контрастности входного изображения.
    ], p=0.3),
    HueSaturationValue(p=0.2), # Произвольное изменение оттенка, насыщенности и значения входного изображения.
], p=1)

#пример
plt.figure(figsize = (12,8))
for i in range(9):
    img = augmentation_3(image = images_train[20])['image']
    plt.subplot(3, 3, i + 1)
    plt.imshow(img)
    plt.axis('off')
plt.show()

In [123]:
augmentation_4 = Compose([
    HorizontalFlip(p=0.2),
    OneOf([
        IAAAdditiveGaussianNoise(p=0.8),
        GaussNoise(p=0.8),
    ], p=0.2),
    OneOf([
        MotionBlur(p=0.9),
        MedianBlur(blur_limit=3, p=0.2),
        Blur(blur_limit=3, p=0.5),
    ], p=0.2),
    ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.5, rotate_limit=10, p=1),
    OneOf([
        OpticalDistortion(p=0.3),
        GridDistortion(p=0.7),
        #PiecewiseAffine(p=0.1),
    ], p=0.7),
    OneOf([
        CLAHE(clip_limit=3), # Примените адаптивное выравнивание гистограммы с ограниченным контрастом к входному изображению.
        IAASharpen(p=0.7), # Увеличьте резкость входного изображения и наложите результат на исходное изображение.
        IAAEmboss(), # Тиснение входного изображения и наложение результата на исходное изображение.
        RandomBrightnessContrast(), # Произвольное изменение яркости и контрастности входного изображения.
    ], p=0.3),
    HueSaturationValue(p=0.6), # Произвольное изменение оттенка, насыщенности и значения входного изображения.
], p=1)

#пример
plt.figure(figsize = (12,8))
for i in range(9):
    img = augmentation_4(image = images_train[20])['image']
    plt.subplot(3, 3, i + 1)
    plt.imshow(img)
    plt.axis('off')
plt.show()

In [ ]:
def make_augmentations(images):
  print('применение аугментаций', end = '')
  augmented_images = np.empty(images.shape)
  for i in range(images.shape[0]):
    if i % 200 == 0:
      print('.', end = '')
    augment_dict = augmentation(image = images[i])
    augmented_image = augment_dict['image']
    augmented_images[i] = augmented_image
  print('')
  return augmented_images

## tf.data.Dataset
Если все изображения мы будем хранить в памяти, то может возникнуть проблема ее нехватки. Не храните все изображения в памяти целиком!

Метод .fit() модели keras может принимать либо данные в виде массивов или тензоров, либо разного рода итераторы, из которых наиболее современным и гибким является [tf.data.Dataset](https://www.tensorflow.org/guide/data). Он представляет собой конвейер, то есть мы указываем, откуда берем данные и какую цепочку преобразований с ними выполняем. Далее мы будем работать с tf.data.Dataset.

Dataset хранит информацию о конечном или бесконечном наборе кортежей (tuple) с данными и может возвращать эти наборы по очереди. Например, данными могут быть пары (input, target) для обучения нейросети. С данными можно осуществлять преобразования, которые осуществляются по мере необходимости ([lazy evaluation](https://ru.wikipedia.org/wiki/%D0%9B%D0%B5%D0%BD%D0%B8%D0%B2%D1%8B%D0%B5_%D0%B2%D1%8B%D1%87%D0%B8%D1%81%D0%BB%D0%B5%D0%BD%D0%B8%D1%8F)).

`tf.data.Dataset.from_tensor_slices(data)` - создает датасет из данных, которые представляют собой либо массив, либо кортеж из массивов. Деление осуществляется по первому индексу каждого массива. Например, если `data = (np.zeros((128, 256, 256)), np.zeros(128))`, то датасет будет содержать 128 элементов, каждый из которых содержит один массив 256x256 и одно число.

`dataset2 = dataset1.map(func)` - применение функции к датасету; функция должна принимать столько аргументов, каков размер кортежа в датасете 1 и возвращать столько, сколько нужно иметь в датасете 2. Пусть, например, датасет содержит изображения и метки, а нам нужно создать датасет только из изображений, тогда мы напишем так: `dataset2 = dataset.map(lambda img, label: img)`.

`dataset2 = dataset1.batch(8)` - группировка по батчам; если датасет 2 должен вернуть один элемент, то он берет из датасета 1 восемь элементов, склеивает их (нулевой индекс результата - номер элемента) и возвращает.

`dataset.__iter__()` - превращение датасета в итератор, из которого можно получать элементы методом `.__next__()`. Итератор, в отличие от самого датасета, хранит позицию текущего элемента. Можно также перебирать датасет циклом for.

`dataset2 = dataset1.repeat(X)` - датасет 2 будет повторять датасет 1 X раз.

Если нам нужно взять из датасета 1000 элементов и использовать их как тестовые, а остальные как обучающие, то мы напишем так:

`test_dataset = dataset.take(1000)
train_dataset = dataset.skip(1000)`

Датасет по сути неизменен: такие операции, как map, batch, repeat, take, skip никак не затрагивают оригинальный датасет. Если датасет хранит элементы [1, 2, 3], то выполнив 3 раза подряд функцию dataset.take(1) мы получим 3 новых датасета, каждый из которых вернет число 1. Если же мы выполним функцию dataset.skip(1), мы получим датасет, возвращающий числа [2, 3], но исходный датасет все равно будет возвращать [1, 2, 3] каждый раз, когда мы его перебираем.

tf.Dataset всегда выполняется в graph-режиме (в противоположность eager-режиму), поэтому либо преобразования (`.map()`) должны содержать только tensorflow-функции, либо мы должны использовать tf.py_function в качестве обертки для функций, вызываемых в `.map()`. Подробнее можно прочитать [здесь](https://www.tensorflow.org/guide/data#applying_arbitrary_python_logic).

In [ ]:
# NLP part
tokenize = Tokenizer(num_words=MAX_WORDS)
tokenize.fit_on_texts(data.description)

In [ ]:
def process_image(image):
    return augmentation_4(image = image.numpy())['image']

def tokenize_(descriptions):
    return sequence.pad_sequences(tokenize.texts_to_sequences(descriptions), maxlen = MAX_SEQUENCE_LENGTH)

def tokenize_text(text):
    return tokenize_([text.numpy().decode('utf-8')])[0]

def tf_process_train_dataset_element(image, table_data, text, price):
    im_shape = image.shape
    [image,] = tf.py_function(process_image, [image], [tf.uint8])
    image.set_shape(im_shape)
    [text,] = tf.py_function(tokenize_text, [text], [tf.int32])
    return (image, table_data, text), price

def tf_process_val_dataset_element(image, table_data, text, price):
    [text,] = tf.py_function(tokenize_text, [text], [tf.int32])
    return (image, table_data, text), price

train_dataset = tf.data.Dataset.from_tensor_slices((
    images_train, X_train, data.description.iloc[X_train.index], y_train
    )).map(tf_process_train_dataset_element)

test_dataset = tf.data.Dataset.from_tensor_slices((
    images_test, X_test, data.description.iloc[X_test.index], y_test
    )).map(tf_process_val_dataset_element)

y_sub = np.zeros(len(X_sub))
sub_dataset = tf.data.Dataset.from_tensor_slices((
    images_sub, X_sub, data.description.iloc[X_sub.index], y_sub
    )).map(tf_process_val_dataset_element)

#проверяем, что нет ошибок (не будет выброшено исключение):
train_dataset.__iter__().__next__();
test_dataset.__iter__().__next__();
sub_dataset.__iter__().__next__();

### Строим сверточную сеть для анализа изображений без "головы"

In [ ]:
#нормализация включена в состав модели EfficientNetB3, поэтому на вход она принимает данные типа uint8
efficientnet_model = tf.keras.applications.efficientnet.EfficientNetB2(weights = 'imagenet', include_top = False, input_shape = (size[1], size[0], 3))
#efficientnet_model = tf.keras.applications.densenet.DenseNet169(weights='imagenet', include_top=False, pooling='avg')
efficientnet_output = L.GlobalAveragePooling2D()(efficientnet_model.output)

In [ ]:
efficientnet_model.trainable = False # Заморозим голову

In [ ]:
model_B4  = Sequential()
model_B4.add(efficientnet_model)
model_B4.add(L.GlobalAveragePooling2D(),)

In [ ]:
model_B4.summary()

In [ ]:
#model_B4.get_config()

In [ ]:
# Моя модель  для табличных данных
tabular_model = Sequential([
    L.Input(shape = X.shape[1]),
    L.Dense(512, activation = para_relu),
    L.Dropout(0.5),
    L.Dense(256, activation = 'relu'),
    L.Dropout(0.5),
    L.Dense(1, activation="linear")
    ])



In [ ]:
# NLP
nlp_model = Sequential([
    L.Input(shape=MAX_SEQUENCE_LENGTH, name="seq_description"),
    L.Embedding(len(tokenize.word_index)+1, MAX_SEQUENCE_LENGTH,),
    L.LSTM(256, return_sequences=True),
    L.Dropout(0.5),
    L.LSTM(128),
    L.Dropout(0.25),
    L.Dense(64),
    ])

In [ ]:
#объединяем выходы трех нейросетей
combinedInput = L.concatenate([model_B4.output, tabular_model.output, nlp_model.output])

# being our regression head
head = L.Dense(256, activation="relu")(combinedInput)
head = L.Dense(1,)(head)

model_3 = Model(inputs=[model_B4.input, tabular_model.input, nlp_model.input], outputs=head)
model_3.summary()

In [ ]:
optimizer = tf.keras.optimizers.Adam(0.0015) #0.002
model_3.compile(loss='MAPE',optimizer=optimizer, metrics=['MAPE'])

In [ ]:
checkpoint = ModelCheckpoint('../working/best_model_nn.hdf5', monitor=['val_MAPE'], verbose=0, mode='min')
earlystop = EarlyStopping(monitor='val_MAPE', patience=40, restore_best_weights=True,)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5,
                             patience=20, min_lr=0.0001)
callbacks_list = [checkpoint, earlystop, reduce_lr]

In [ ]:
history = model_3.fit(train_dataset.batch(30), #30
                    epochs=150,
                    validation_data = test_dataset.batch(30), # 30
                    callbacks=callbacks_list
                   )

In [ ]:
plt.title('Loss')
plt.plot(history.history['MAPE'], label='train')
plt.plot(history.history['val_MAPE'], label='test')
plt.show();

In [ ]:
model_3.load_weights('../working/best_model_nn.hdf5')
model_3.save('../working/model_nn.hdf5')

In [ ]:
test_predict_nn3 = model_3.predict(test_dataset.batch(30))
print(f"TEST mape: {(mape(y_test, test_predict_nn3[:,0]))*100:0.2f}%")

In [ ]:
test_predict_nn3 = model.predict(test_dataset.batch(30))
print(f"TEST mape: {(mape(y_test, test_predict_nn3[:,0]))*100:0.2f}%")

In [ ]:
sub_predict_nn3 = model_3.predict(sub_dataset.batch(30))
sample_submission['price'] = sub_predict_nn3[:,0]
sample_submission.to_csv('nn3_submission.csv', index=False)

In [ ]:
# Активируем "головы"

efficientnet_model.trainable = True
fun_tune_model = len(efficientnet_model.layers) // 2
for layers in efficientnet_model.layers[:fun_tune_model]:
    layers.trainable = False

In [ ]:
# Строим модель

fn_model  = Sequential()
fn_model.add(efficientnet_model)
fn_model.add(L.GlobalAveragePooling2D(),)

In [ ]:
# Моя модель  для табличных данных
tabular_model_f = Sequential([
    L.Input(shape = X.shape[1]),
    L.Dense(512, activation = para_relu),
    L.Dropout(0.5),
    L.Dense(256, activation = 'relu'),
    L.Dropout(0.5),
    L.Dense(1, activation="linear")
    ])

In [ ]:
# NLP
nlp_model_f = Sequential([
    L.Input(shape=MAX_SEQUENCE_LENGTH, name="seq_description"),
    L.Embedding(len(tokenize.word_index)+1, MAX_SEQUENCE_LENGTH,),
    L.LSTM(256, return_sequences=True),
    L.Dropout(0.5),
    L.LSTM(128),
    L.Dropout(0.25),
    L.Dense(64),
    ])

In [ ]:
#объединяем выходы трех нейросетей
combinedInput_f = L.concatenate([fn_model.output, tabular_model_f.output, nlp_model_f.output])

# being our regression head
head_f = L.Dense(256, activation="relu")(combinedInput_f)
head_f = L.Dense(1,)(head_f)

model_nn = Model(inputs=[fn_model.input, tabular_model_f.input, nlp_model_f.input], outputs=head_f)
model_nn.summary()

In [ ]:
optimizer = tf.keras.optimizers.Adam(0.0015) #0.002
model_nn.compile(loss='MAPE',optimizer=optimizer, metrics=['MAPE'])

In [ ]:
checkpoint = ModelCheckpoint('../working/best_model_nn_2.hdf5', monitor=['val_MAPE'], verbose=0, mode='min')
earlystop = EarlyStopping(monitor='val_MAPE', patience=40, restore_best_weights=True,)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5,
                             patience=20, min_lr=0.00001)
callbacks_list = [checkpoint, earlystop, reduce_lr]

In [ ]:
history = model_nn.fit(train_dataset.batch(30), #30
                    epochs=200,
                    validation_data = test_dataset.batch(30), # 30
                    callbacks=callbacks_list
                   )

In [ ]:
plt.title('Loss')
plt.plot(history.history['MAPE'], label='train')
plt.plot(history.history['val_MAPE'], label='test')
plt.show();

In [ ]:
model_nn.load_weights('../working/best_model_nn4.hdf5')
model_nn.save('../working/model_nn4.hdf5')

In [ ]:
test_predict_nn4 = model_nn.predict(test_dataset.batch(30))
print(f"TEST mape: {(mape(y_test, test_predict_nn3[:,0]))*100:0.2f}%")

In [ ]:
sub_predict_nn4 = model_nn.predict(sub_dataset.batch(30))
sample_submission['price'] = sub_predict_nn4[:,0]
sample_submission.to_csv('nn4_submission.csv', index=False)

In [ ]:
#нормализация включена в состав модели EfficientNetB3, поэтому на вход она принимает данные типа uint8
efficientnet_model1 = tf.keras.applications.efficientnet.EfficientNetB3(weights = 'imagenet', include_top = False, input_shape = (size[1], size[0], 3))
#efficientnet_model = tf.keras.applications.densenet.DenseNet169(weights='imagenet', include_top=False, pooling='avg')
efficientnet_output = L.GlobalAveragePooling2D()(efficientnet_model1.output)

In [ ]:
efficientnet_model1.trainable = False # Заморозим голову

In [ ]:
model_B4  = Sequential()
model_B4.add(efficientnet_model1)
model_B4.add(L.GlobalAveragePooling2D(),)

In [ ]:
# Моя модель  для табличных данных
tabular_model = Sequential([
    L.Input(shape = X.shape[1]),
    L.Dense(512, activation = para_relu),
    L.Dropout(0.5),
    L.Dense(256, activation = 'relu'),
    L.Dropout(0.5),
    L.Dense(1, activation="linear")
    ])


In [ ]:
# NLP
nlp_model = Sequential([
    L.Input(shape=MAX_SEQUENCE_LENGTH, name="seq_description"),
    L.Embedding(len(tokenize.word_index)+1, MAX_SEQUENCE_LENGTH,),
    L.LSTM(256, return_sequences=True),
    L.Dropout(0.5),
    L.LSTM(128),
    L.Dropout(0.25),
    L.Dense(64),
    ])

In [ ]:
#объединяем выходы трех нейросетей
combinedInput = L.concatenate([model_B4.output, tabular_model.output, nlp_model.output])

# being our regression head
head = L.Dense(256, activation="relu")(combinedInput)
head = L.Dense(1,)(head)

model_2 = Model(inputs=[model_B4.input, tabular_model.input, nlp_model.input], outputs=head)
model_2.summary()

In [ ]:
optimizer = tf.keras.optimizers.Adam(0.002)
model_2.compile(loss='MAPE',optimizer=optimizer, metrics=['MAPE'])

In [ ]:
history = model_2.fit(train_dataset.batch(30), #30
                    epochs=300,
                    validation_data = test_dataset.batch(30), # 30
                    callbacks=callbacks_list
                   )

Тут попробовал поиграть со скоростью, аугментацией, немного увеличивал размер картинки, менял модели (EfficientNetB3, EfficientNetB2, EfficientNetB6) увеличивал кол-во эпох, понижал batch,пробовал замораживать размораживать слои, но результата менее 12% у меня достичь не получилось. Можно было конечно подольше поэкспериментировать с моделями и аугментацией, применить fine-tuning, но 30ч  с GPU, которые даются на Kaggle на неделю этого очень мало, а на Colab нет возможности сейчас продлить подписку на Pro, да и скачать с Kaggle картинки не дает. 
!kaggle competitions download -c sf-dst-car-price-prediction-part2 показывает ошибку 403 - Forbidden

По результату у меня получилось добиться результата 10.87% за счет обработки табличных данных и текста


#### Общие рекомендации:
* Попробовать разные архитектуры
* Провести более детальный анализ результатов
* Попробовать различные подходы в управление LR и оптимизаторы
* Поработать с таргетом
* Использовать Fine-tuning

#### Tabular
* В нейросеть желательно подавать данные с распределением, близким к нормальному, поэтому от некоторых числовых признаков имеет смысл взять логарифм перед нормализацией. Пример:
`modelDateNorm = np.log(2020 - data['modelDate'])`
Статья по теме: https://habr.com/ru/company/ods/blog/325422

* Извлечение числовых значений из текста:
Парсинг признаков 'engineDisplacement', 'enginePower', 'Владение' для извлечения числовых значений.

* Cокращение размерности категориальных признаков
Признак name 'name' содержит данные, которые уже есть в других столбцах ('enginePower', 'engineDisplacement', 'vehicleTransmission'). Можно удалить эти данные. Затем можно еще сильнее сократить размерность, например выделив наличие xDrive в качестве отдельного признака.

* Поработать над Feature engineering



#### NLP
* Выделить из описаний часто встречающиеся блоки текста, заменив их на кодовые слова или удалив
* Сделать предобработку текста, например сделать лемматизацию - алгоритм ставящий все слова в форму по умолчанию (глаголы в инфинитив и т. д.), чтобы токенайзер не преобразовывал разные формы слова в разные числа
Статья по теме: https://habr.com/ru/company/Voximplant/blog/446738/
* Поработать над алгоритмами очистки и аугментации текста



#### CV
* Попробовать различные аугментации
* Fine-tuning

# Blend

## Попробуем комбинацию из результатов предсказаний предыдущих моделей

In [ ]:
test_predict_nn4 = model_nn.predict(test_dataset.batch(30))

In [ ]:
#test_predict_nn2 = model_2.predict(test_dataset.batch(30))

In [ ]:
blend_predict = (test_predict_catboost + test_predict_nn2[:,0]) / 2
print(f"TEST mape: {(mape(y_test, blend_predict))*100:0.2f}%")

In [ ]:
#blend_predict = (sub_predict_catboost + test_predict_nn2[:,0]) / 2
sample_submission['price'] = blend_predict
sample_submission.to_csv('blend_submission.csv', index=False)

# Model Bonus: проброс признака

In [ ]:
# MLP
model_mlp = Sequential()
model_mlp.add(L.Dense(512, input_dim=X_train.shape[1], activation="relu"))
model_mlp.add(L.Dropout(0.5))
model_mlp.add(L.Dense(256, activation="relu"))
model_mlp.add(L.Dropout(0.5))

In [ ]:
# FEATURE Input
# Iput
productiondate = L.Input(shape=[1], name="productiondate")
# Embeddings layers
emb_productiondate = L.Embedding(len(X.productionDate.unique().tolist())+1, 20)(productiondate)
f_productiondate = L.Flatten()(emb_productiondate)

In [ ]:
combinedInput = L.concatenate([model_mlp.output, f_productiondate,])
# being our regression head
head = L.Dense(64, activation="relu")(combinedInput)
head = L.Dense(1, activation="linear")(head)

model = Model(inputs=[model_mlp.input, productiondate], outputs=head)

In [ ]:
model.summary()

In [ ]:
optimizer = tf.keras.optimizers.Adam(0.01)
model.compile(loss='MAPE',optimizer=optimizer, metrics=['MAPE'])

In [ ]:
history = model.fit([X_train, X_train.productionDate.values], y_train,
                    batch_size=512,
                    epochs=500, # фактически мы обучаем пока EarlyStopping не остановит обучение
                    validation_data=([X_test, X_test.productionDate.values], y_test),
                    callbacks=callbacks_list
                   )

In [ ]:
model.load_weights('../working/best_model.hdf5')
test_predict_nn_bonus = model.predict([X_test, X_test.productionDate.values])
print(f"TEST mape: {(mape(y_test, test_predict_nn_bonus[:,0]))*100:0.2f}%")

In [ ]:
checkpoint = ModelCheckpoint('../working/best_model_nn_2.hdf5', monitor=['val_MAPE'], verbose=0, mode='min')
earlystop = EarlyStopping(monitor='val_MAPE', patience=40, restore_best_weights=True,)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5,
                             patience=20, min_lr=0.00001)
callbacks_list = [checkpoint, earlystop, reduce_lr]

In [ ]:
# 